# 오산시 어린이 교통사고 위험지역 도출

## 1. Data 불러오기

In [1]:
#12/24 GeoSeries 추가
from geoband.API import *
import pandas as pd
import folium
import json
import geopandas as gpd
from geojson import Feature, FeatureCollection, Point, dump
from geopandas import GeoSeries
from shapely.geometry import Point
import matplotlib.pyplot as plt
import pydeck as pdk
import shapely
import numpy as np
plt.rc("font", family = "Malgun Gothic")
%matplotlib inline
import warnings
warnings.filterwarnings(action='ignore') 
from sklearn.linear_model import LinearRegression
from sklearn import linear_model

In [2]:
GetCompasData('SBJ_2012_001', '1', '1.오산시_주정차단속(2018~2020).csv')
GetCompasData('SBJ_2012_001', '2', '2.오산시_어린이교통사고_격자.geojson')
GetCompasData('SBJ_2012_001', '3', '3.오산시_차량등록현황_격자.geojson')
GetCompasData('SBJ_2012_001', '4', '4.오산시_연령별_거주인구격자(총인구).geojson')
GetCompasData('SBJ_2012_001', '5', '5.오산시_연령별_거주인구격자(유소년).geojson')
GetCompasData('SBJ_2012_001', '6', '6.오산시_연령별_거주인구격자(생산가능인구).geojson')
GetCompasData('SBJ_2012_001', '7', '7.오산시_연령별_거주인구격자(고령).geojson')
GetCompasData('SBJ_2012_001', '8', '8.오산시_유동인구(2019).csv')
GetCompasData('SBJ_2012_001', '9', '9.오산시_어린이보호구역.csv')
GetCompasData('SBJ_2012_001', '10', '10.오산시_학교위치정보.csv')
GetCompasData('SBJ_2012_001', '11', '11.오산시_초등학교_통학구.geojson')
GetCompasData('SBJ_2012_001', '12', '12.오산시_중학교_학군.geojson')
GetCompasData('SBJ_2012_001', '13', '13.오산시_어린이집_유치원현황.csv')
GetCompasData('SBJ_2012_001', '14', '14.오산시_기상데이터(2010~2019).csv')
GetCompasData('SBJ_2012_001', '15', '15.오산시_무인교통단속카메라.csv')
GetCompasData('SBJ_2012_001', '16', '16.오산시_도로안전표지표준데이터.csv')
GetCompasData('SBJ_2012_001', '17', '17.오산시_횡단보도.geojson')
GetCompasData('SBJ_2012_001', '18', '18.오산시_과속방지턱표준데이터.csv')
GetCompasData('SBJ_2012_001', '19', '19.오산시_신호등.geojson')
GetCompasData('SBJ_2012_001', '20', '20.오산시_CCTV설치현황.csv')
GetCompasData('SBJ_2012_001', '21', '21.오산시_인도.geojson')
GetCompasData('SBJ_2012_001', '22', '22.오산시_버스정류장.csv')
GetCompasData('SBJ_2012_001', '23', '23.오산시_상세도로망_LV6.geojson')
GetCompasData('SBJ_2012_001', '24', '24.평일_전일,시간대별_오산시_추정교통량_Level6.csv')
GetCompasData('SBJ_2012_001', '25', '25.평일_전일_오산시_혼잡빈도강도_Level6.csv')
GetCompasData('SBJ_2012_001', '26', '26.평일_전일_오산시_혼잡시간강도_Level6.csv')
GetCompasData('SBJ_2012_001', '27', '27.오산시_도로명주소_건물.geojson')
GetCompasData('SBJ_2012_001', '28', '28.오산시_건물연면적_격자.geojson')
GetCompasData('SBJ_2012_001', '29', '29.오산시_체육시설현황.csv')
GetCompasData('SBJ_2012_001', '30', '30.오산시_학원_및_교습소_현황.csv')
GetCompasData('SBJ_2012_001', '31', '31.오산시_법정경계(시군구).geojson')
GetCompasData('SBJ_2012_001', '32', '32.오산시_행정경계(읍면동).geojson')
GetCompasData('SBJ_2012_001', '33', '33.오산시_법정경계(읍면동).geojson')
GetCompasData('SBJ_2012_001', '34', '34.오산시_지적도.geojson')

In [3]:
df1_parking=pd.read_csv('1.오산시_주정차단속(2018~2020).csv')
df2_grid_accident=gpd.read_file('2.오산시_어린이교통사고_격자.geojson')
df3_grid_car_enrollment=gpd.read_file('3.오산시_차량등록현황_격자.geojson')
df4_pop=gpd.read_file('4.오산시_연령별_거주인구격자(총인구).geojson')
df5_junior_pop=gpd.read_file('5.오산시_연령별_거주인구격자(유소년).geojson')
df6_product_pop=gpd.read_file('6.오산시_연령별_거주인구격자(생산가능인구).geojson')
df7_senior_pop=gpd.read_file('7.오산시_연령별_거주인구격자(고령).geojson')
df8_floating_pop=pd.read_csv('8.오산시_유동인구(2019).csv')
df9_protection_area=pd.read_csv('9.오산시_어린이보호구역.csv')
df10_school=pd.read_csv('10.오산시_학교위치정보.csv')
df11_elementary_district=gpd.read_file('11.오산시_초등학교_통학구.geojson')
df12_middle_district=gpd.read_file('12.오산시_중학교_학군.geojson')
df13_kinder=pd.read_csv('13.오산시_어린이집_유치원현황.csv')
df14_weather=pd.read_csv('14.오산시_기상데이터(2010~2019).csv')
df15_traffic_cctv=pd.read_csv('15.오산시_무인교통단속카메라.csv')
df16_road_sign=pd.read_csv('16.오산시_도로안전표지표준데이터.csv')
df17_crosswalk=gpd.read_file('17.오산시_횡단보도.geojson')
df18_speed_bump=pd.read_csv('18.오산시_과속방지턱표준데이터.csv')
df19_traffic_light=gpd.read_file('19.오산시_신호등.geojson')
df20_cctv=pd.read_csv('20.오산시_CCTV설치현황.csv')
df21_sidewalk=gpd.read_file('21.오산시_인도.geojson')
df22_bus_stop=pd.read_csv('22.오산시_버스정류장.csv')
df23_road=gpd.read_file('23.오산시_상세도로망_LV6.geojson')
df24_traffic=pd.read_csv('24.평일_전일,시간대별_오산시_추정교통량_Level6.csv')
df25_traffic_frequency=pd.read_csv('25.평일_전일_오산시_혼잡빈도강도_Level6.csv')
df26_traffic_time=pd.read_csv('26.평일_전일_오산시_혼잡시간강도_Level6.csv')
df27_building=gpd.read_file('27.오산시_도로명주소_건물.geojson')
df28_grid_building=gpd.read_file('28.오산시_건물연면적_격자.geojson')
df29_sports=pd.read_csv('29.오산시_체육시설현황.csv')
df30_academy=pd.read_csv('30.오산시_학원_및_교습소_현황.csv')
df31_le_sig=gpd.read_file('31.오산시_법정경계(시군구).geojson')
df32_admin_emd=gpd.read_file('32.오산시_행정경계(읍면동).geojson')
df33_legal_emd=gpd.read_file('33.오산시_법정경계(읍면동).geojson')

In [4]:
# 좌표계 설정
df1_parking.crs = {'init':'epsg:4326'}
df2_grid_accident.crs = {'init':'epsg:4326'}
df3_grid_car_enrollment.crs = {'init':'epsg:4326'}
df4_pop.crs = {'init':'epsg:4326'}
df5_junior_pop.crs = {'init':'epsg:4326'}
df6_product_pop.crs = {'init':'epsg:4326'}
df7_senior_pop.crs = {'init':'epsg:4326'}
df8_floating_pop.crs = {'init':'epsg:4326'}
df9_protection_area.crs = {'init':'epsg:4326'}
df10_school.crs = {'init':'epsg:4326'}
df11_elementary_district.crs = {'init':'epsg:4326'}
df12_middle_district.crs = {'init':'epsg:4326'}
df13_kinder.crs = {'init':'epsg:4326'}
df14_weather.crs = {'init':'epsg:4326'}
df15_traffic_cctv.crs = {'init':'epsg:4326'}
df16_road_sign.crs = {'init':'epsg:4326'}
df17_crosswalk.crs = {'init':'epsg:4326'}
df18_speed_bump.crs = {'init':'epsg:4326'}
df19_traffic_light.crs = {'init':'epsg:4326'}
df20_cctv.crs = {'init':'epsg:4326'}
df21_sidewalk.crs = {'init':'epsg:4326'}
df22_bus_stop.crs = {'init':'epsg:4326'}
df23_road.crs = {'init':'epsg:4326'}
df24_traffic.crs = {'init':'epsg:4326'}
df25_traffic_frequency.crs = {'init':'epsg:4326'}
df26_traffic_time.crs = {'init':'epsg:4326'}
df27_building.crs = {'init':'epsg:4326'}
df28_grid_building.crs = {'init':'epsg:4326'}
df29_sports.crs = {'init':'epsg:4326'}
df30_academy.crs = {'init':'epsg:4326'}
df31_le_sig.crs = {'init':'epsg:4326'}
df32_admin_emd.crs = {'init':'epsg:4326'}
df33_legal_emd.crs = {'init':'epsg:4326'}

## 2. 분석 기초 함수

### 1) pydeck 사용을 위해 geometry의 Point, Multilinestring, Multipolygon을 coordinates로 바꿔주기 위한 함수 선언

In [5]:
# 12/25 창균 추가
def POINT_to_coordinates(geo_data):
    geo_data['lat'] = geo_data['geometry'].apply(lambda coord: coord.y)
    geo_data['lon'] = geo_data['geometry'].apply(lambda coord: coord.x)
    return geo_data

# 사용 예
# df19_traffic_light = POINT_to_coordinates(df19_traffic_light)

In [6]:
# 12/25 창균 추가
def MULTILINESTRING_to_coordinates(line_string):
    if isinstance(line_string, shapely.geometry.linestring.LineString):
        lon, lat = line_string.xy
        return [[x, y] for x, y in zip(lon, lat)]
    elif isinstance(line_string, shapely.geometry.multilinestring.MultiLineString):
        ret = []
        for i in range(len(line_string)):
            lon, lat = line_string[i].xy
            for x, y in zip(lon, lat):
                ret.append([x, y])
        return ret
    
# 사용 예
# df23_road['coordinates'] = df23_road['geometry'].apply(MULTILINESTRING_to_coordinates)
# df23_road = pd.DataFrame(df23_road) # geopanadas 가 아닌 pandas 의 데이터프레임으로 꼭 바꿔줘야 합니다.

In [7]:
# 12/25 창균 추가
def MULTIPOLYGON_to_coordinates(x):
    lon, lat = x[0].exterior.xy
    return [[x, y] for x, y in zip(lon, lat)]

# 사용 예
# df_osan_grid['coordinates'] = df_osan_grid['geometry'].apply(MULTIPOLYGON_to_coordinates)
# df_osan_grid = pd.DataFrame(df_osan_grid) # geopanadas 가 아닌 pandas 의 데이터프레임으로 꼭 바꿔줘야 합니다.

### 2) 각각의 df의 lon, lat을 geometry의 point로 변환

In [8]:
# 12/24 정민 수정사항
def location_to_point(df):
    point_df = gpd.points_from_xy(df.lon, df.lat)
    point_df = GeoSeries(point_df)
    
    col = df.columns
    
    loc_df = gpd.GeoDataFrame(df[col], geometry = point_df)
    
    return loc_df

### 3) sjoin : 격자내 존재하는 좌표 개수의 합을 구하는 함수

In [9]:
def sjoin(grid_info,df, new_col_name, option = 'contains'):
    joined= gpd.sjoin(grid_info,df, op=option)
    result = joined.groupby('gid').size()
    result = result.to_frame().reset_index()
    grid_info=pd.merge(grid_info,result, how='outer',on='gid')
    
    grid_info = grid_info.rename(columns = {0:new_col_name})
    grid_info=grid_info.fillna(0)
    
    return grid_info

## 3. 목적에 맞는 데이터 전처리

### 기본 그리드 생성

In [10]:
#지도에 맵핑할 격자들 만들기
# 12/25 창균 추가
df_osan_grid=df4_pop.drop('val',axis=1)

df_osan_grid.head(5)

,gid,geometry
0,다사551085,"MULTIPOLYGON (((126.99422 37.17418, 126.99421 ..."
1,다사551086,"MULTIPOLYGON (((126.99421 37.17508, 126.99420 ..."
2,다사551087,"MULTIPOLYGON (((126.99420 37.17599, 126.99420 ..."
3,다사551088,"MULTIPOLYGON (((126.99420 37.17689, 126.99419 ..."
4,다사552085,"MULTIPOLYGON (((126.99534 37.17419, 126.99534 ..."


### df1 오산시 주정차 단속

In [11]:
#df1 오산시 주정차 단속
# 12/25 창균 추가
df1_parking.columns=['date','parking_location','admin_area','school_zone',
                 'lon','lat','parking_type']

#스쿨존에 해당하면 1 아니면 0
df1_parking.loc[df1_parking['school_zone']=='N','school_zone']=0
df1_parking.loc[df1_parking['school_zone']=='Y','school_zone']=1

# 오산시 유탑유블레스 남측 좌표 삽입 (널 처리)
df1_parking["lon"].fillna(127.072291, inplace = True)
df1_parking["lat"].fillna(37.141363, inplace = True)

df1_parking = location_to_point(df1_parking)
df1_parking.head()

,date,parking_location,admin_area,school_zone,lon,lat,parking_type,geometry
0,20181101,금암동 여계산로 부근,경기도 오산시 세교동,0,127.043462,37.170237,이동형CCTV단속,POINT (127.04346 37.17024)
1,20181101,세교동 세마역로19번길,경기도 오산시 세교동,0,127.044770,37.181695,이동형CCTV단속,POINT (127.04477 37.18169)
2,20181101,세교동 수목원로 부근,경기도 오산시 양산동,0,127.042200,37.186357,이동형CCTV단속,POINT (127.04220 37.18636)
3,20181101,오산동 오산로272번길 부,경기도 오산시 원동,0,127.071732,37.150557,이동형CCTV단속,POINT (127.07173 37.15056)
4,20181101,오산동 성호대로93번길,경기도 오산시 부산동,0,127.071757,37.150457,이동형CCTV단속,POINT (127.07176 37.15046)


### df4 인구, df5 유소년 인구, df6 생산가능인구, df7 노인 인구 - 인구수 변수 이름 변경 및 널처리

In [12]:
#나중에 비율 구할때 써야됨
#val값에 null은 측정이 안된 곳이기에 0으로 처리해줌-SY
# 12/25 창균 추가
df4_pop=df4_pop.rename(columns={'val':'pop_val'})
df4_pop=df4_pop.fillna(0)

df5_junior_pop=df5_junior_pop.rename(columns={'val':'junior_val'})
df5_junior_pop=df5_junior_pop.fillna(0)

df6_product_pop=df6_product_pop.rename(columns={'val':'product_val'})
df6_product_pop=df6_product_pop.fillna(0)

df7_senior_pop=df7_senior_pop.rename(columns={'val':'senior_val'})
df7_senior_pop=df7_senior_pop.fillna(0)

In [13]:
# 정민 수정사항(12/21)
# 오산시 격자별 유소년 인구수/총 인구수 데이터 전처리
df_junior_ratio_pop = df5_junior_pop.merge(df4_pop, on ='gid')
df_junior_ratio_pop = df_junior_ratio_pop.drop(['geometry_x'], axis = 1)
df_junior_ratio_pop.columns = ['gid', 'junior_val', 'pop_val', 'geometry']

# 비율 컬럼 생성
df_junior_ratio_pop['junior_ratio'] = df_junior_ratio_pop['junior_val']/df_junior_ratio_pop['pop_val'] * 100
df_junior_ratio_pop['junior_ratio'].value_counts()
df_junior_ratio_pop=df_junior_ratio_pop.fillna(0)

# # geometry to coordinates
# df_junior_ratio_pop['coordinates'] = df_junior_ratio_pop['geometry'].apply(MULTIPOLYGON_to_coordinates)
# # df_junior_ratio_pop = pd.DataFrame(df_junior_ratio_pop)

df_junior_ratio_pop

,gid,junior_val,pop_val,geometry,junior_ratio
0,다사551085,0.0,0.0,"MULTIPOLYGON (((126.99422 37.17418, 126.99421 ...",0.0
1,다사551086,0.0,0.0,"MULTIPOLYGON (((126.99421 37.17508, 126.99420 ...",0.0
2,다사551087,0.0,0.0,"MULTIPOLYGON (((126.99420 37.17599, 126.99420 ...",0.0
3,다사551088,0.0,0.0,"MULTIPOLYGON (((126.99420 37.17689, 126.99419 ...",0.0
4,다사552085,0.0,0.0,"MULTIPOLYGON (((126.99534 37.17419, 126.99534 ...",0.0
...,...,...,...,...,...
4505,다사646052,0.0,0.0,"MULTIPOLYGON (((127.10138 37.14484, 127.10138 ...",0.0
4506,다사646053,0.0,0.0,"MULTIPOLYGON (((127.10138 37.14575, 127.10137 ...",0.0
4507,다사647050,0.0,0.0,"MULTIPOLYGON (((127.10252 37.14305, 127.10251 ...",0.0
4508,다사647051,0.0,0.0,"MULTIPOLYGON (((127.10251 37.14395, 127.10251 ...",0.0


### df8 유동인구

In [14]:
#합치는 코드 있어서 한번만 돌려야됨
#유동인구
df_col_list=list(df8_floating_pop)
df_col_list_all=df_col_list[3:]
df_col_list_8_23=df_col_list[11:26]
df_col_list_14_20=df_col_list[17:23]

#모든 시간대 유동인구
df8_floating_pop['all']=df8_floating_pop[df_col_list_all].sum(axis=1)

#아이들의 이동시간
df8_floating_pop['8_23']=df8_floating_pop[df_col_list_8_23].sum(axis=1)

#사고가 가장 많이 일어나는 14~20
df8_floating_pop['14_20']=df8_floating_pop[df_col_list_14_20].sum(axis=1)
df8_floating_pop=df8_floating_pop.rename(columns={'STD_YM':'YM','lon':'lon','lat':'lat'})

#lat, lon to point & pd.DataFrame to gpd.GeodataFrame
df8_floating_pop = location_to_point(df8_floating_pop)

df8_floating_pop

,YM,lon,lat,TMST_00,TMST_01,TMST_02,TMST_03,TMST_04,TMST_05,TMST_06,...,TMST_18,TMST_19,TMST_20,TMST_21,TMST_22,TMST_23,all,8_23,14_20,geometry
0,201901,126.995145,37.176703,0.08,0.05,0.04,0.04,0.08,0.27,0.80,...,0.70,0.43,0.34,0.22,0.14,0.10,11.70,9.30,4.44,POINT (126.99515 37.17670)
1,201901,126.995142,37.177153,0.06,0.04,0.03,0.03,0.06,0.19,0.56,...,0.51,0.33,0.26,0.17,0.11,0.08,8.88,7.11,3.31,POINT (126.99514 37.17715)
2,201901,126.995714,37.175804,0.00,0.00,0.00,0.00,0.00,0.01,0.03,...,0.03,0.02,0.02,0.01,0.01,0.00,0.58,0.49,0.23,POINT (126.99571 37.17580)
3,201901,126.995711,37.176254,0.05,0.03,0.02,0.03,0.05,0.19,0.56,...,0.48,0.29,0.23,0.15,0.10,0.07,8.08,6.43,3.06,POINT (126.99571 37.17625)
4,201901,126.995708,37.176705,0.02,0.01,0.01,0.01,0.02,0.07,0.18,...,0.17,0.11,0.09,0.06,0.04,0.03,3.08,2.49,1.12,POINT (126.99571 37.17670)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125158,201912,127.097213,37.154107,0.09,0.07,0.05,0.04,0.04,0.04,0.05,...,0.14,0.14,0.14,0.14,0.12,0.11,2.37,1.81,0.75,POINT (127.09721 37.15411)
125159,201912,127.097819,37.145997,0.23,0.18,0.12,0.09,0.09,0.11,0.14,...,0.34,0.35,0.35,0.34,0.32,0.28,5.86,4.43,1.89,POINT (127.09782 37.14600)
125160,201912,127.097793,37.150955,0.02,0.02,0.02,0.02,0.02,0.02,0.02,...,0.04,0.04,0.04,0.04,0.04,0.04,0.78,0.58,0.24,POINT (127.09779 37.15095)
125161,201912,127.098354,37.151407,0.02,0.02,0.02,0.00,0.00,0.00,0.02,...,0.04,0.04,0.04,0.04,0.02,0.02,0.50,0.38,0.16,POINT (127.09835 37.15141)


### df9 어린이 보호구역

In [15]:
#어린이 보호구역
df9_protection_area.columns=['protection_type','protection_name','protection_cctv',
                         'protection_cctv_count','width','lon','lat']
#cctv 있으면 1 없으면 0
df9_protection_area.loc[df9_protection_area['protection_cctv']=='N','protection_cctv']=0
df9_protection_area.loc[df9_protection_area['protection_cctv']=='Y','protection_cctv']=1

# object typecasting to int
df9_protection_area['protection_cctv'] = df9_protection_area['protection_cctv'].astype('int')

#lat, lon to point & pd.DataFrame to gpd.GeodataFrame
df9_protection_area = location_to_point(df9_protection_area)

df9_protection_area

,protection_type,protection_name,protection_cctv,protection_cctv_count,width,lon,lat,geometry
0,특수학교,성심학교,0,0,25,127.017521,37.176762,POINT (127.01752 37.17676)
1,초등학교,가수초등학교,1,1,12,127.047957,37.148103,POINT (127.04796 37.14810)
2,초등학교,광성초등학교,1,2,12,127.039709,37.184197,POINT (127.03971 37.18420)
3,초등학교,대호초등학교,1,3,12,127.057889,37.159106,POINT (127.05789 37.15911)
4,초등학교,매홀초등학교,1,2,6,127.063282,37.162749,POINT (127.06328 37.16275)
...,...,...,...,...,...,...,...,...
86,어린이집,시립세마어린이집,1,1,12,127.044916,37.184608,POINT (127.04492 37.18461)
87,어린이집,미래숲어린이집,1,1,6,127.044384,37.145326,POINT (127.04438 37.14533)
88,어린이집,시립은여울어린이집,0,0,20,127.066860,37.170952,POINT (127.06686 37.17095)
89,어린이집,설리반어린이집,0,0,5,127.061041,37.164127,POINT (127.06104 37.16413)


### df10 학교 위치정보

In [16]:
# 학교 위치정보
# 중학교와 고등학교는 비대상으로 drop 처리
a=df10_school.loc[df10_school['학교구분'].isin(['고등학교','중학교'])].index
df10_school=df10_school.drop(a)
df10_school.columns=['school_type','school_name','lon','lat']
df10_school = df10_school.reset_index(drop=True)

#lat, lon to point & pd.DataFrame to gpd.GeodataFrame
df10_school = location_to_point(df10_school)

df10_school

,school_type,school_name,lon,lat,geometry
0,초등학교,원동초등학교,127.069699,37.135168,POINT (127.06970 37.13517)
1,초등학교,운산초등학교,127.077414,37.152500,POINT (127.07741 37.15250)
2,초등학교,화성초등학교,127.068858,37.164615,POINT (127.06886 37.16461)
3,초등학교,가수초등학교,127.047749,37.147919,POINT (127.04775 37.14792)
4,초등학교,다온초등학교,127.090752,37.151698,POINT (127.09075 37.15170)
5,초등학교,매홀초등학교,127.063587,37.162365,POINT (127.06359 37.16237)
6,초등학교,문시초등학교,127.045862,37.174141,POINT (127.04586 37.17414)
7,초등학교,성산초등학교,127.066424,37.149695,POINT (127.06642 37.14969)
8,초등학교,세미초등학교,127.064666,37.173997,POINT (127.06467 37.17400)
9,초등학교,양산초등학교,127.032111,37.196887,POINT (127.03211 37.19689)


### df11 초등학교 통학구

In [17]:
# 초등학교 통학구
# type이 0이면 단독 통학구 1이면 다중 통학구로 예상됨
df11_elementary_district.columns=['ed_objectid','district_id','ed_name','ed_type','geometry']

# object typecasting to int
df11_elementary_district['ed_type'] = df11_elementary_district['ed_type'].astype('int')
df11_elementary_district.head()

,ed_objectid,district_id,ed_name,ed_type,geometry
0,2648,Z000102542,오산고현초통학구역,0,"MULTIPOLYGON (((127.08560 37.13923, 127.08593 ..."
1,2774,Z000102531,수청초통학구역,0,"MULTIPOLYGON (((127.06076 37.17523, 127.06074 ..."
2,3259,Z000150216,대호초성산초공동통학구역,1,"MULTIPOLYGON (((127.03921 37.16869, 127.03953 ..."
3,3300,Z000102507,문시초통학구역,0,"MULTIPOLYGON (((127.05068 37.17408, 127.05084 ..."
4,3299,Z000102567,필봉초통학구역,0,"MULTIPOLYGON (((127.05151 37.18063, 127.05274 ..."


### df13 어린이집 유치원 현황

In [18]:
# 어린이집 유치원현황
df13_kinder=df13_kinder.drop('시설타입',axis=1)
df13_kinder.columns=['kinder_type','kinder_name','lon','lat']

#lat, lon to point & pd.DataFrame to gpd.GeodataFrame
df13_kinder = location_to_point(df13_kinder)

df13_kinder

,kinder_type,kinder_name,lon,lat,geometry
0,어린이집,해오름어린이집,127.084336,37.132383,POINT (127.08434 37.13238)
1,어린이집,햇살가득어린이집,127.041817,37.182164,POINT (127.04182 37.18216)
2,어린이집,행복한아이들어린이집,127.079494,37.152891,POINT (127.07949 37.15289)
3,어린이집,휴튼사과나무어린이집,127.049638,37.177976,POINT (127.04964 37.17798)
4,어린이집,(주)아모레퍼시픽 오산어린이집,127.040252,37.165311,POINT (127.04025 37.16531)
...,...,...,...,...,...
302,유치원,매홀초등학교병설유치원,127.063587,37.162365,POINT (127.06359 37.16237)
303,유치원,문시초등학교병설유치원,127.045862,37.174141,POINT (127.04586 37.17414)
304,유치원,세교유치원,127.044332,37.177051,POINT (127.04433 37.17705)
305,유치원,수청초등학교병설유치원,127.055667,37.170668,POINT (127.05567 37.17067)


### df15 무인교통 단속 카메라

In [19]:
#무인교통 단속 카메라
df15_traffic_cctv=df15_traffic_cctv.drop(['도로노선방향','단속구분'],axis=1)
df15_traffic_cctv.columns=['tc_road_name','tc_location_name','lon','lat']

#lat, lon to point & pd.DataFrame to gpd.GeodataFrame
df15_traffic_cctv = location_to_point(df15_traffic_cctv)

df15_traffic_cctv

,tc_road_name,tc_location_name,lon,lat,geometry
0,궐리사로,궐동-법원앞(법원사거리),127.059357,37.159313,POINT (127.05936 37.15931)
1,역광장로,원동-오산역광장,127.067796,37.145409,POINT (127.06780 37.14541)
2,궐리사로,궐동-제일하이빌,127.062386,37.159198,POINT (127.06239 37.15920)
3,궐리사로,궐리사 주차장 앞,127.061920,37.158559,POINT (127.06192 37.15856)
4,운암로,부산동-운천초교 입구,127.077991,37.156216,POINT (127.07799 37.15622)
...,...,...,...,...,...
107,경기동로186번길,시티자이2차 아파트 앞,127.091941,37.154632,POINT (127.09194 37.15463)
108,역광장로,오원사거리,127.068303,37.142519,POINT (127.06830 37.14252)
109,청학로,차량등록사업소,127.064287,37.158197,POINT (127.06429 37.15820)
110,수목원로468번길,세교9단지 후문 상가도로,127.050714,37.175949,POINT (127.05071 37.17595)


### df16 도로안전표지 표준 데이터

In [20]:
#도로안전표지 표준데이터
df16_road_sign.columns=['road_sign_num','road_sign_type','lon','lat']

#lat, lon to point & pd.DataFrame to gpd.GeodataFrame
df16_road_sign = location_to_point(df16_road_sign)

df16_road_sign

,road_sign_num,road_sign_type,lon,lat,geometry
0,ER-400[봉담동탄고속도로]-상-60,2방향예고표지,127.069227,37.182663,POINT (127.06923 37.18266)
1,NR-1[경기대로]-하-1982,시/군계표지,127.040826,37.197859,POINT (127.04083 37.19786)
2,UR(오산시)-16[남부대로]-상-5,3방향도로명예고표지,127.070216,37.135929,POINT (127.07022 37.13593)
3,UR(오산시)-16[남부대로]-하-3,3방향표지,127.064316,37.135531,POINT (127.06432 37.13553)
4,UR(오산시)-16[남부대로]-하-4,3방향예고표지,127.051717,37.134248,POINT (127.05172 37.13425)
...,...,...,...,...,...
826,UR(오산시)-[부원로]-하-2,2방향표지,127.084624,37.144540,POINT (127.08462 37.14454)
827,RR-310[가장로]-상-170,2방향예고표지,127.035840,37.160577,POINT (127.03584 37.16058)
828,RR-82[성호대로]-하-51,사설표지(허가),127.064309,37.149266,POINT (127.06431 37.14927)
829,RR-82[성호대로]-하-52,하천표지,127.062244,37.149482,POINT (127.06224 37.14948)


### df17 횡단보도

In [21]:
#횡단보도
## 창균 추가 수정 (12.22)
df17_crosswalk.columns=['crosswalk_objectid','geometry']
df17_crosswalk

,crosswalk_objectid,geometry
0,2,"MULTIPOLYGON (((127.06170 37.15704, 127.06178 ..."
1,3,"MULTIPOLYGON (((127.03174 37.19499, 127.03176 ..."
2,4,"MULTIPOLYGON (((127.05525 37.15645, 127.05533 ..."
3,5,"MULTIPOLYGON (((127.04935 37.13434, 127.04939 ..."
4,6,"MULTIPOLYGON (((127.06216 37.14973, 127.06228 ..."
...,...,...
1200,1292,"MULTIPOLYGON (((127.07846 37.13971, 127.07842 ..."
1201,1294,"MULTIPOLYGON (((127.08997 37.15087, 127.08994 ..."
1202,1295,"MULTIPOLYGON (((127.05778 37.15212, 127.05792 ..."
1203,1297,"MULTIPOLYGON (((127.04281 37.18184, 127.04291 ..."


### df18 과속방지턱 표준데이터

In [22]:
#과속방지턱표준데이터
df18_speed_bump.columns=['bump_road_name','bump_location_name','bump_height','bump_width',
                   'bump_length','bump_sep','bump_continue','lon','lat']
#과속방지턱 보차분리여부,연속형 여부 0,1로 변환
df18_speed_bump.loc[df18_speed_bump['bump_sep']=='N','bump_sep']=0
df18_speed_bump.loc[df18_speed_bump['bump_sep']=='Y','bump_sep']=1
df18_speed_bump.loc[df18_speed_bump['bump_continue']=='N','bump_continue']=0
df18_speed_bump.loc[df18_speed_bump['bump_continue']=='Y','bump_continue']=1

# object typecasting to int
df18_speed_bump['bump_sep'] = df18_speed_bump['bump_sep'].astype('int')
df18_speed_bump['bump_continue'] = df18_speed_bump['bump_continue'].astype('int')

#lat, lon to point & pd.DataFrame to gpd.GeodataFrame
df18_speed_bump = location_to_point(df18_speed_bump)

df18_speed_bump

,bump_road_name,bump_location_name,bump_height,bump_width,bump_length,bump_sep,bump_continue,lon,lat,geometry
0,오산로,소망빌 앞,10,3.6,5.4,0,1,127.068954,37.130852,POINT (127.06895 37.13085)
1,동부대로,태영철강재공장 앞,10,3.6,5.0,0,1,127.076305,37.137946,POINT (127.07630 37.13795)
2,밀머리로,메디마트 앞,0,3.6,5.4,1,1,127.075093,37.141466,POINT (127.07509 37.14147)
3,오산로,다인하우스 앞,10,3.6,10.0,1,1,127.071169,37.143833,POINT (127.07117 37.14383)
4,운천로,운천고등학교 앞,10,3.6,6.0,1,1,127.080466,37.155872,POINT (127.08047 37.15587)
5,성호대로,오산우체국 앞,10,3.6,5.0,1,1,127.070781,37.149169,POINT (127.07078 37.14917)
6,경기대로,오산동 어린이공원 앞,10,3.6,6.0,0,1,127.072608,37.151294,POINT (127.07261 37.15129)
7,청학로,오산중학교 앞,10,3.6,6.5,1,1,127.061185,37.155124,POINT (127.06118 37.15512)
8,청학로,세산다세대 부근,10,3.6,4.3,1,0,127.062582,37.152656,POINT (127.06258 37.15266)
9,청학로,정석빌딩 앞,10,3.6,4.4,1,0,127.062101,37.150484,POINT (127.06210 37.15048)


### df19 신호등

In [23]:
#신호등
## 창균 추가 수정 (12.22)
## 창균 추가 수정 (12/25)
df19_traffic_light.columns=['traffic_light_objectid','geometry']

# geometry to coordinates
df19_traffic_light = POINT_to_coordinates(df19_traffic_light)

# geometry to coordinates
df19_traffic_light = POINT_to_coordinates(df19_traffic_light)
# df19_traffic_light = pd.DataFrame(df19_traffic_light)

df19_traffic_light

,traffic_light_objectid,geometry,lat,lon
0,1,POINT (127.07160 37.14407),37.144067,127.071599
1,2,POINT (127.08149 37.12849),37.128490,127.081488
2,3,POINT (127.03978 37.16155),37.161546,127.039777
3,4,POINT (127.03999 37.16168),37.161682,127.039990
4,5,POINT (127.07838 37.14296),37.142955,127.078383
...,...,...,...,...
1493,1494,POINT (127.07888 37.15909),37.159089,127.078879
1494,1495,POINT (127.07832 37.15993),37.159930,127.078323
1495,1496,POINT (127.07850 37.15977),37.159766,127.078502
1496,1497,POINT (127.07820 37.15971),37.159706,127.078201


### df20 cctv

In [24]:
#cctv
# location_to_point 함수에서 위도 경도를 point로 가져오려면 각 df의 특성에 맞는 이름인 df_lat, df_lon가 아니라 lat, lon으로 정해준다.
# cctv_type A: 방범 B:도시공원 C:어린이보호 D:차량방범
df20_cctv.columns=['cctv_type','cctv_road_name','lat','lon']
#cctv_lat,lon에서 중복적으로 결측값이 나와서 drop, road_name도 null값 nan으로 처리-SY
df20_cctv=df20_cctv.fillna('NaN')

#lat, lon to point & pd.DataFrame to gpd.GeodataFrame
df20_cctv = location_to_point(df20_cctv)

df20_cctv

,cctv_type,cctv_road_name,lat,lon,geometry
0,A,성호대로55번길 31-2,37.151836,127.067541,POINT (127.068 37.152)
1,A,성호대로93번길 45,37.152812,127.071702,POINT (127.072 37.153)
2,C,성호대로93번길 26,37.151167,127.071677,POINT (127.072 37.151)
3,A,운천로165번길 52-1,37.157164,127.075991,POINT (127.076 37.157)
4,A,운천로165번길 5,37.156461,127.077273,POINT (127.077 37.156)
...,...,...,...,...,...
562,E,성호초,127.070001,37.149096,POINT (37.149 127.070)
563,E,대원초,127.072728,37.131781,POINT (37.132 127.073)
564,E,원일초,127.071237,37.139445,POINT (37.139 127.071)
565,E,운천초,127.078048,37.156239,POINT (37.156 127.078)


### df21 인도

In [25]:
#인도
df21_sidewalk=df21_sidewalk.drop('QUAL',axis=1)
df21_sidewalk.columns=['sidewalk_id','sidewalk_bicycle',
                  'sidewalk_type','geometry']
#자전거 도로유무 BYC001이 있고 나머지가 없음
df21_sidewalk.loc[df21_sidewalk['sidewalk_bicycle']=='BYC001','sidewalk_bicycle']=1
df21_sidewalk.loc[df21_sidewalk['sidewalk_bicycle']=='BYC002','sidewalk_bicycle']=0

# object typecasting to int
df21_sidewalk['sidewalk_bicycle'] = df21_sidewalk['sidewalk_bicycle'].astype('int')

df21_sidewalk

,sidewalk_id,sidewalk_bicycle,sidewalk_type,geometry
0,TRN0400000002N7T0,1,SWK001,"MULTILINESTRING ((127.05144 37.14183, 127.0514..."
1,TRN0400000002N79G,1,SWK001,"MULTILINESTRING ((127.04907 37.14108, 127.0490..."
2,TRN0400000002N7LS,1,SWK001,"MULTILINESTRING ((127.05210 37.14163, 127.0521..."
3,TRN0400000002NAR1,1,SWK001,"MULTILINESTRING ((127.07471 37.14870, 127.0747..."
4,TRN0400000003E7DC,1,SWK001,"MULTILINESTRING ((127.07466 37.14988, 127.0746..."
...,...,...,...,...
1335,TRN050000000048WK,1,SWK002,"MULTILINESTRING ((127.04644 37.17758, 127.0463..."
1336,TRN050000000048XL,1,SWK002,"MULTILINESTRING ((127.02588 37.17715, 127.0259..."
1337,TRN050000000048YM,1,SWK002,"MULTILINESTRING ((127.02783 37.17885, 127.0278..."
1338,TRN050000000048ZN,1,SWK002,"MULTILINESTRING ((127.04136 37.19749, 127.0413..."


### df22 버스 정류장

In [26]:
# 버스 정류장
# location_to_point 함수에서 위도 경도를 point로 가져오려면 각 df의 특성에 맞는 이름인 df_lat, df_lon가 아니라 lat, lon으로 정해준다.
## 창균 추가 수정 (12.22)
df22_bus_stop.columns=['bus_stop_id','bus_stop_name','lon','lat']

#lat, lon to point & pd.DataFrame to gpd.GeodataFrame
df22_bus_stop = location_to_point(df22_bus_stop)

df22_bus_stop

,bus_stop_id,bus_stop_name,lon,lat,geometry
0,223000095,오산시티자이1.2단지,127.091033,37.149983,POINT (127.09103 37.14998)
1,223000124,재래시장앞,127.068700,37.152717,POINT (127.06870 37.15272)
2,223000290,솔매마을,127.026650,37.178833,POINT (127.02665 37.17883)
3,223000289,솔매마을,127.026617,37.178933,POINT (127.02662 37.17893)
4,223000306,죽미마을입구,127.046117,37.178217,POINT (127.04612 37.17822)
...,...,...,...,...,...
407,223000501,한신대학교,127.023550,37.196083,POINT (127.02355 37.19608)
408,223000384,성운시스템,127.058383,37.191317,POINT (127.05838 37.19132)
409,223000268,대우푸르지오,127.082333,37.136200,POINT (127.08233 37.13620)
410,223000468,오산마을.한국마트,127.080317,37.135817,POINT (127.08032 37.13582)


### df23 상세 도로망

In [27]:
#상세도로망
#road_name null값 nan으로 처리-SY
# 고속도로 제거
df23_road=df23_road[df23_road['road_rank'].isin(['103','105','106','107'])]
# 불필요한 컬럼 제거
df23_road=df23_road.drop('facil_name',axis=1)
# 널값 처리
df23_road=df23_road.fillna('NaN')

# object typecasting to int
df23_road = df23_road.apply(pd.to_numeric, errors = 'ignore')
# 뒤에 교통량 분석을 위해 id는 str으로 타입 캐스팅
df23_road['link_id'] = df23_road['link_id'].astype(str)

# 인덱스 초기화
df23_road.reset_index(drop=True, inplace=True)

# df23_road.columns
# gdf_col = ['link_id', 'max_speed', 'road_name', 'road_no', 'road_rank',
#        'link_type', 'road_type', 'up_lanes', 'dw_lanes', 'oneway', 'length',
#        'width', 'car_lane', 'num_cross', 'barrier',]

# gdf_road = gpd.GeoDataFrame(df23_road[gdf_col], geometry= df23_road.geometry)
# gdf_road

df23_road

,link_id,max_speed,road_name,road_no,road_rank,link_type,road_type,up_lanes,dw_lanes,oneway,length,width,car_lane,num_cross,barrier,geometry
0,571200632,0,성호대로,0,107,4,0,1,0,1,0.060,1,0,0,0,"MULTILINESTRING ((127.06151 37.14928, 127.0615..."
1,571200633,0,황새로,0,107,4,0,1,0,1,0.057,1,0,0,0,"MULTILINESTRING ((127.06096 37.14943, 127.0611..."
2,571200680,60,오산로,0,107,32768,0,1,1,0,0.017,2,0,0,0,"MULTILINESTRING ((127.06835 37.15484, 127.0682..."
3,571200686,60,대원로,0,107,32768,0,2,2,0,0.073,3,0,0,0,"MULTILINESTRING ((127.07356 37.14637, 127.0729..."
4,571200693,30,경기대로,0,107,32768,0,1,1,0,0.038,2,0,0,0,"MULTILINESTRING ((127.07215 37.13379, 127.0725..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1807,571372254,0,가장산업서북로,0,107,32768,0,2,2,0,0.024,3,0,0,15,"MULTILINESTRING ((127.02656 37.16962, 127.0267..."
1808,571372255,0,NaN,0,107,32768,0,2,2,0,0.057,3,0,0,0,"MULTILINESTRING ((127.03183 37.17111, 127.0318..."
1809,571372256,0,NaN,0,107,32768,0,2,2,0,0.011,3,0,0,0,"MULTILINESTRING ((127.03191 37.17172, 127.0318..."
1810,900000166,0,NaN,0,107,32768,0,1,0,1,0.065,1,0,0,0,"MULTILINESTRING ((127.05992 37.16942, 127.0601..."


### df24 평일 시간대별 추정 교통량

In [28]:
#평일 시간대별 추정교통량
# road_name null값 nan으로 처리-SY
# 고속도로 제외
df24_traffic=df24_traffic[df24_traffic['도로등급'].isin(['103','105','106','107'])]
# 필요없는 컬럼 삭제
df24_traffic=df24_traffic.drop(['평일주말','시도명','시군구명'],axis=1)
# 널값 NaN 처리
df24_traffic=df24_traffic.fillna('NaN')

df24_traffic.columns=['link_id','road_rank','link_length','road_name',
               'emd_name','time','all_traffic','car_traffic',
                'bus_traffic','freight_car_traffic']
#traffic의 time column에서 dtype이 통일이 되어 있지 않아 str로 통일함-SY
df24_traffic['time']=df24_traffic['time'].apply(str)

# 인덱스 초기화
df24_traffic.reset_index(drop=True, inplace=True)

df24_traffic

,link_id,road_rank,link_length,road_name,emd_name,time,all_traffic,car_traffic,bus_traffic,freight_car_traffic
0,57124529301,107,0.030,NaN,대원동,0,2.02,1.92,0.00,0.10
1,57124529302,107,0.030,NaN,대원동,0,1.51,1.27,0.00,0.24
2,57124362301,107,0.031,NaN,대원동,0,5.72,4.19,0.00,1.53
3,57124362302,107,0.031,NaN,대원동,0,2.80,2.47,0.00,0.33
4,57124364001,107,0.031,NaN,대원동,0,3.64,3.52,0.01,0.10
...,...,...,...,...,...,...,...,...,...,...
80595,90000016701,107,0.063,NaN,신장동,19,20.41,20.29,0.01,0.12
80596,90000016701,107,0.063,NaN,신장동,20,16.16,16.08,0.01,0.07
80597,90000016701,107,0.063,NaN,신장동,21,12.80,12.75,0.00,0.05
80598,90000016701,107,0.063,NaN,신장동,22,8.92,8.90,0.00,0.02


### df25 평일 혼잡빈도강도

In [29]:
# 평일 혼잡빈도강도
#road_name null값 nan으로 처리-SY
df25_traffic_frequency=df25_traffic_frequency[df25_traffic_frequency['도로등급'].isin(['103','105','106','107'])]
df25_traffic_frequency=df25_traffic_frequency.drop(['시도명','시군구명','평일주말','시간적범위'],axis=1)
df25_traffic_frequency=df25_traffic_frequency.fillna('NaN')
df25_traffic_frequency.columns=['link_id','road_rank','link_length','road_name',
                          'emd_name','frequency']
df25_traffic_frequency['link_id'] = df25_traffic_frequency['link_id'].astype(str)

# 인덱스 초기화
df25_traffic_frequency.reset_index(drop=True, inplace=True)

df25_traffic_frequency

,link_id,road_rank,link_length,road_name,emd_name,frequency
0,57125206201,103,0.007,경기대로,대원동,98.60
1,57124864802,103,0.007,경기대로,대원동,99.18
2,57124864801,103,0.007,경기대로,대원동,99.88
3,57124436602,103,0.010,경기대로,대원동,98.51
4,57124436601,103,0.010,경기대로,대원동,99.16
...,...,...,...,...,...,...
2999,57124331702,105,0.107,경기동로,중앙동,92.50
3000,57124331701,105,0.107,경기동로,중앙동,94.17
3001,57125313201,105,0.115,경기동로,중앙동,65.47
3002,90000016701,107,0.063,NaN,신장동,100.00


### df26 평일 혼잡시간강도

In [30]:
#평일 혼잡시간강도
#road_name null값 nan으로 처리-SY
df26_traffic_time=df26_traffic_time[df26_traffic_time['도로등급'].isin(['103','105','106','107'])]
df26_traffic_time=df26_traffic_time.drop(['시도명','시군구명','평일주말','시간적범위'],axis=1)
df26_traffic_time=df26_traffic_time.fillna('NaN')
df26_traffic_time.columns=['link_id','road_rank','link_length','road_name',
                          'emd_name','time']
df26_traffic_time['link_id'] = df26_traffic_time['link_id'].astype(str)

# 인덱스 초기화
df26_traffic_time.reset_index(drop=True, inplace=True)

df26_traffic_time

,link_id,road_rank,link_length,road_name,emd_name,time
0,57125206201,103,0.007,경기대로,대원동,99.36
1,57124864802,103,0.007,경기대로,대원동,99.53
2,57124864801,103,0.007,경기대로,대원동,99.95
3,57124436602,103,0.010,경기대로,대원동,99.61
4,57124436601,103,0.010,경기대로,대원동,99.66
...,...,...,...,...,...,...
2999,57124063002,106,0.359,남부대로,초평동,23.01
3000,57125193601,106,0.489,남부대로,초평동,80.72
3001,57125193602,106,0.489,남부대로,초평동,90.84
3002,57124902602,106,0.660,가장로,초평동,71.30


### df27 도로명주소 건물

In [31]:
# 창균 수정 (12/26)
#건물
# 불필요한 컬럼 삭제
df27_building=df27_building.drop(['UND_FLO_CO','GRO_FLO_CO','BULD_NM_DC','BULD_NM'],axis=1)

# object typecasting to int
df27_building = df27_building.apply(pd.to_numeric, errors = 'ignore')

df27_building

,BDTYP_CD,BULD_SE_CD,BUL_MAN_NO,EMD_CD,LNBR_MNNM,LNBR_SLNO,geometry
0,4001,0,23692,102,454,5,"MULTIPOLYGON (((127.08714 37.15535, 127.08714 ..."
1,21005,0,18171,112,519,2,"MULTIPOLYGON (((127.01085 37.18895, 127.01067 ..."
2,21005,0,18154,114,58,1,"MULTIPOLYGON (((127.03588 37.17902, 127.03680 ..."
3,15006,0,18491,114,58,1,"MULTIPOLYGON (((127.03548 37.17860, 127.03576 ..."
4,21005,0,18170,107,42,1,"MULTIPOLYGON (((127.03867 37.17610, 127.03795 ..."
...,...,...,...,...,...,...,...
16444,17105,0,11004,116,607,0,"MULTIPOLYGON (((127.03249 37.14693, 127.03364 ..."
16445,17105,0,11007,116,607,0,"MULTIPOLYGON (((127.03331 37.14699, 127.03406 ..."
16446,17203,0,11008,116,607,0,"MULTIPOLYGON (((127.03347 37.14644, 127.03356 ..."
16447,17105,0,11005,116,607,0,"MULTIPOLYGON (((127.03253 37.14660, 127.03310 ..."


### df28 격자별 건물 연면적

In [32]:
# 창균 수정 (12/26)
#격자별 건물연면적
#val값의 null은 측정이 안된 값이라 0으로 처리-SY
df28_grid_building.columns=['gid','grid_building_val','geometry']
df28_grid_building=df28_grid_building.fillna(0)

df28_grid_building

,gid,grid_building_val,geometry
0,다사551085,0.0,"MULTIPOLYGON (((126.99422 37.17418, 126.99421 ..."
1,다사551086,0.0,"MULTIPOLYGON (((126.99421 37.17508, 126.99420 ..."
2,다사551087,0.0,"MULTIPOLYGON (((126.99420 37.17599, 126.99420 ..."
3,다사551088,0.0,"MULTIPOLYGON (((126.99420 37.17689, 126.99419 ..."
4,다사552085,0.0,"MULTIPOLYGON (((126.99534 37.17419, 126.99534 ..."
...,...,...,...
4505,다사646052,0.0,"MULTIPOLYGON (((127.10138 37.14484, 127.10138 ..."
4506,다사646053,0.0,"MULTIPOLYGON (((127.10138 37.14575, 127.10137 ..."
4507,다사647050,0.0,"MULTIPOLYGON (((127.10252 37.14305, 127.10251 ..."
4508,다사647051,0.0,"MULTIPOLYGON (((127.10251 37.14395, 127.10251 ..."


### df29 체육시설

In [33]:
#체육시설
df29_sports.columns=['sports_type','lon','lat']

#lat, lon to point & pd.DataFrame to gpd.GeodataFrame
df29_sports = location_to_point(df29_sports)

df29_sports

,sports_type,lon,lat,geometry
0,육상시설,127.073804,37.157197,POINT (127.07380 37.15720)
1,기타시설,127.041923,37.179440,POINT (127.04192 37.17944)
2,기타시설,127.075720,37.158754,POINT (127.07572 37.15875)
3,테니스,127.077445,37.149806,POINT (127.07744 37.14981)
4,테니스,127.070223,37.155640,POINT (127.07022 37.15564)
...,...,...,...,...
140,체육도장업,127.045700,37.147668,POINT (127.04570 37.14767)
141,체육도장업,127.056167,37.160918,POINT (127.05617 37.16092)
142,체육도장업,127.068877,37.150437,POINT (127.06888 37.15044)
143,체육도장업,127.090695,37.150806,POINT (127.09069 37.15081)


### df30 학원 교습소

In [34]:
#학원,교습소
df30_academy=df30_academy.drop('교습과정명',axis=1)
df30_academy.columns=['academy_type','lon','lat']
df30_academy=df30_academy.fillna('NaN')

#lat, lon to point & pd.DataFrame to gpd.GeodataFrame
df30_academy = location_to_point(df30_academy)

# 평생직업교육학원 제거 후
del_index = df30_academy[df30_academy['academy_type'] == '평생직업교육학원'].index
df30_academy = df30_academy.drop(del_index)
# 인덱스 초기화
df30_academy.reset_index(drop=True, inplace=True)

df30_academy

,academy_type,lon,lat,geometry
0,교습소,127.048919,37.146279,POINT (127.04892 37.14628)
1,교습소,127.048919,37.146279,POINT (127.04892 37.14628)
2,교습소,127.052419,37.145809,POINT (127.05242 37.14581)
3,학교교과교습학원,127.048919,37.146279,POINT (127.04892 37.14628)
4,학교교과교습학원,127.048919,37.146279,POINT (127.04892 37.14628)
...,...,...,...,...
1675,학교교과교습학원,127.050280,37.145999,POINT (127.05028 37.14600)
1676,학교교과교습학원,127.049558,37.142116,POINT (127.04956 37.14212)
1677,학교교과교습학원,127.062722,37.165033,POINT (127.06272 37.16503)
1678,학교교과교습학원,127.062722,37.165033,POINT (127.06272 37.16503)


# 4. 분석을 위한 추가 처리

## 1) 격차별 데이터

In [35]:
#각 격자에 교통사고 횟수 concat
df_osan_grid_info=pd.concat([df_osan_grid,df2_grid_accident['accident_cnt']],axis=1)
#각 격자에 거주인구수 concat
df_osan_grid_info=pd.concat([df_osan_grid_info,df4_pop['pop_val']],axis=1)
#각 격자에 유소년 거주인구수 concat
df_osan_grid_info=pd.concat([df_osan_grid_info,df5_junior_pop['junior_val']],axis=1)
#각 격자에 건물 면적 concat
df_osan_grid_info=pd.concat([df_osan_grid_info,df28_grid_building['grid_building_val']],axis=1)
df_osan_grid_info

,gid,geometry,accident_cnt,pop_val,junior_val,grid_building_val
0,다사551085,"MULTIPOLYGON (((126.99422 37.17418, 126.99421 ...",0,0.0,0.0,0.0
1,다사551086,"MULTIPOLYGON (((126.99421 37.17508, 126.99420 ...",0,0.0,0.0,0.0
2,다사551087,"MULTIPOLYGON (((126.99420 37.17599, 126.99420 ...",0,0.0,0.0,0.0
3,다사551088,"MULTIPOLYGON (((126.99420 37.17689, 126.99419 ...",0,0.0,0.0,0.0
4,다사552085,"MULTIPOLYGON (((126.99534 37.17419, 126.99534 ...",0,0.0,0.0,0.0
...,...,...,...,...,...,...
4505,다사646052,"MULTIPOLYGON (((127.10138 37.14484, 127.10138 ...",0,0.0,0.0,0.0
4506,다사646053,"MULTIPOLYGON (((127.10138 37.14575, 127.10137 ...",0,0.0,0.0,0.0
4507,다사647050,"MULTIPOLYGON (((127.10252 37.14305, 127.10251 ...",0,0.0,0.0,0.0
4508,다사647051,"MULTIPOLYGON (((127.10251 37.14395, 127.10251 ...",0,0.0,0.0,0.0


## 2) 교통량 분석

In [36]:
# 14~20시에 해당하는 교통량과 상세도로망의 데이터 merge
df_traffic_1420=df24_traffic[df24_traffic['time'].isin(['14','15','16','17,''18','19'])]
df_traffic_road = []
for i in df23_road['link_id']:
    df_traffic_road.append([i,sum(df_traffic_1420[df_traffic_1420['link_id'].apply(str).str.contains(i)]['all_traffic']),
                         sum(df_traffic_1420[df_traffic_1420['link_id'].apply(str).str.contains(i)]['freight_car_traffic'])])
df_traffic_road=pd.DataFrame(df_traffic_road)       
df_traffic_road.columns=['link_id','all_traffic','freight_car_traffic']

#road_info에 모든 도로의 교통량,시간복잡도,빈도복잡도의 데이터를 merge한다
df_road_info=pd.merge(df23_road,df_traffic_road,on='link_id')
df_road_info

,link_id,max_speed,road_name,road_no,road_rank,link_type,road_type,up_lanes,dw_lanes,oneway,length,width,car_lane,num_cross,barrier,geometry,all_traffic,freight_car_traffic
0,571200632,0,성호대로,0,107,4,0,1,0,1,0.060,1,0,0,0,"MULTILINESTRING ((127.06151 37.14928, 127.0615...",0.00,0.00
1,571200633,0,황새로,0,107,4,0,1,0,1,0.057,1,0,0,0,"MULTILINESTRING ((127.06096 37.14943, 127.0611...",45.59,1.56
2,571200680,60,오산로,0,107,32768,0,1,1,0,0.017,2,0,0,0,"MULTILINESTRING ((127.06835 37.15484, 127.0682...",3640.57,340.84
3,571200686,60,대원로,0,107,32768,0,2,2,0,0.073,3,0,0,0,"MULTILINESTRING ((127.07356 37.14637, 127.0729...",1992.60,195.29
4,571200693,30,경기대로,0,107,32768,0,1,1,0,0.038,2,0,0,0,"MULTILINESTRING ((127.07215 37.13379, 127.0725...",753.16,114.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1807,571372254,0,가장산업서북로,0,107,32768,0,2,2,0,0.024,3,0,0,15,"MULTILINESTRING ((127.02656 37.16962, 127.0267...",1269.17,313.73
1808,571372255,0,NaN,0,107,32768,0,2,2,0,0.057,3,0,0,0,"MULTILINESTRING ((127.03183 37.17111, 127.0318...",566.93,128.03
1809,571372256,0,NaN,0,107,32768,0,2,2,0,0.011,3,0,0,0,"MULTILINESTRING ((127.03191 37.17172, 127.0318...",356.34,97.69
1810,900000166,0,NaN,0,107,32768,0,1,0,1,0.065,1,0,0,0,"MULTILINESTRING ((127.05992 37.16942, 127.0601...",146.04,1.07


## 3) 평일시간복잡도

In [37]:
# 평일시간복잡도와 상세도로망 merge
df_traffic_time_road = []
for i in df23_road['link_id']:
    df_traffic_time_road.append([i,sum(df26_traffic_time[df26_traffic_time['link_id'].apply(str).str.contains(i)]['time'])])
    
df_traffic_time_road=pd.DataFrame(df_traffic_time_road)
df_traffic_time_road.columns=['link_id','time']
# road_info에 시간복잡도 merge
df_road_info=pd.merge(df_road_info,df_traffic_time_road,on='link_id')
df_road_info

,link_id,max_speed,road_name,road_no,road_rank,link_type,road_type,up_lanes,dw_lanes,oneway,length,width,car_lane,num_cross,barrier,geometry,all_traffic,freight_car_traffic,time
0,571200632,0,성호대로,0,107,4,0,1,0,1,0.060,1,0,0,0,"MULTILINESTRING ((127.06151 37.14928, 127.0615...",0.00,0.00,0.00
1,571200633,0,황새로,0,107,4,0,1,0,1,0.057,1,0,0,0,"MULTILINESTRING ((127.06096 37.14943, 127.0611...",45.59,1.56,96.47
2,571200680,60,오산로,0,107,32768,0,1,1,0,0.017,2,0,0,0,"MULTILINESTRING ((127.06835 37.15484, 127.0682...",3640.57,340.84,180.76
3,571200686,60,대원로,0,107,32768,0,2,2,0,0.073,3,0,0,0,"MULTILINESTRING ((127.07356 37.14637, 127.0729...",1992.60,195.29,163.51
4,571200693,30,경기대로,0,107,32768,0,1,1,0,0.038,2,0,0,0,"MULTILINESTRING ((127.07215 37.13379, 127.0725...",753.16,114.57,196.53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1807,571372254,0,가장산업서북로,0,107,32768,0,2,2,0,0.024,3,0,0,15,"MULTILINESTRING ((127.02656 37.16962, 127.0267...",1269.17,313.73,149.12
1808,571372255,0,NaN,0,107,32768,0,2,2,0,0.057,3,0,0,0,"MULTILINESTRING ((127.03183 37.17111, 127.0318...",566.93,128.03,174.95
1809,571372256,0,NaN,0,107,32768,0,2,2,0,0.011,3,0,0,0,"MULTILINESTRING ((127.03191 37.17172, 127.0318...",356.34,97.69,198.65
1810,900000166,0,NaN,0,107,32768,0,1,0,1,0.065,1,0,0,0,"MULTILINESTRING ((127.05992 37.16942, 127.0601...",146.04,1.07,100.00


## 4) 평일빈도복잡도

In [38]:
# 평일빈도복잡도와 상세도로망 merge
df25_traffic_frequency_road = []
for i in df23_road['link_id']:
    df25_traffic_frequency_road.append([i,sum(df25_traffic_frequency[df25_traffic_frequency['link_id'].apply(str).str.contains(i)]['frequency'])])
df25_traffic_frequency_road=pd.DataFrame(df25_traffic_frequency_road)     
df25_traffic_frequency_road.columns=['link_id','frequency']
#road_info에 빈도복잡도 merge
df_road_info=pd.merge(df_road_info,df25_traffic_frequency_road,on='link_id')
df_road_info

,link_id,max_speed,road_name,road_no,road_rank,link_type,road_type,up_lanes,dw_lanes,oneway,length,width,car_lane,num_cross,barrier,geometry,all_traffic,freight_car_traffic,time,frequency
0,571200632,0,성호대로,0,107,4,0,1,0,1,0.060,1,0,0,0,"MULTILINESTRING ((127.06151 37.14928, 127.0615...",0.00,0.00,0.00,0.00
1,571200633,0,황새로,0,107,4,0,1,0,1,0.057,1,0,0,0,"MULTILINESTRING ((127.06096 37.14943, 127.0611...",45.59,1.56,96.47,90.53
2,571200680,60,오산로,0,107,32768,0,1,1,0,0.017,2,0,0,0,"MULTILINESTRING ((127.06835 37.15484, 127.0682...",3640.57,340.84,180.76,149.19
3,571200686,60,대원로,0,107,32768,0,2,2,0,0.073,3,0,0,0,"MULTILINESTRING ((127.07356 37.14637, 127.0729...",1992.60,195.29,163.51,122.32
4,571200693,30,경기대로,0,107,32768,0,1,1,0,0.038,2,0,0,0,"MULTILINESTRING ((127.07215 37.13379, 127.0725...",753.16,114.57,196.53,184.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1807,571372254,0,가장산업서북로,0,107,32768,0,2,2,0,0.024,3,0,0,15,"MULTILINESTRING ((127.02656 37.16962, 127.0267...",1269.17,313.73,149.12,114.33
1808,571372255,0,NaN,0,107,32768,0,2,2,0,0.057,3,0,0,0,"MULTILINESTRING ((127.03183 37.17111, 127.0318...",566.93,128.03,174.95,132.83
1809,571372256,0,NaN,0,107,32768,0,2,2,0,0.011,3,0,0,0,"MULTILINESTRING ((127.03191 37.17172, 127.0318...",356.34,97.69,198.65,195.12
1810,900000166,0,NaN,0,107,32768,0,1,0,1,0.065,1,0,0,0,"MULTILINESTRING ((127.05992 37.16942, 127.0601...",146.04,1.07,100.00,100.00


# 5.전처리된 데이터 저장소에 저장

In [39]:
df_osan_grid.to_file('df_osan_grid.geojson', driver="GeoJSON")
df_road_info.to_file('df_road_info.geojson', driver="GeoJSON")

df1_parking.to_file('df1_parking.geojson', driver="GeoJSON")
df2_grid_accident.to_file('df2_grid_accident.geojson', driver="GeoJSON")
df4_pop.to_file('df4_pop.geojson', driver="GeoJSON")
df5_junior_pop.to_file('df5_junior_pop.geojson', driver="GeoJSON")
df6_product_pop.to_file('df6_product_pop.geojson', driver="GeoJSON")
df7_senior_pop.to_file('df7_senior_pop.geojson', driver="GeoJSON")
df8_floating_pop.to_file('df8_floating_pop.geojson', driver="GeoJSON")
df9_protection_area.to_file('df9_protection_area.geojson', driver="GeoJSON")
df10_school.to_file('df10_school.geojson', driver="GeoJSON")
df11_elementary_district.to_file('df11_elementary_district.geojson', driver="GeoJSON")
df12_middle_district.to_file('df12_middle_district.geojson', driver="GeoJSON")
df13_kinder.to_file('df13_kinder.geojson', driver="GeoJSON")
df15_traffic_cctv.to_file('df15_traffic_cctv.geojson', driver="GeoJSON")
df16_road_sign.to_file('df16_road_sign.geojson', driver="GeoJSON")
df17_crosswalk.to_file('df17_crosswalk.geojson', driver="GeoJSON")
df18_speed_bump.to_file('df18_speed_bump.geojson', driver="GeoJSON")
df19_traffic_light.to_file('df19_traffic_light.geojson', driver="GeoJSON")
df20_cctv.to_file('df20_cctv.geojson', driver="GeoJSON")
df21_sidewalk.to_file('df21_sidewalk.geojson', driver="GeoJSON")
df22_bus_stop.to_file('df22_bus_stop.geojson', driver="GeoJSON")

# explicitly set it as geometry for the GeoDataFrame
df23_road.set_geometry(col='geometry', inplace=True)
df23_road.to_file('df23_road.geojson', driver="GeoJSON")

df24_traffic.to_csv('df24_traffic.csv', encoding = 'utf-8', index = False)
df25_traffic_frequency.to_csv('df25_traffic_frequency.csv', encoding = 'utf-8', index = False)
df26_traffic_time.to_csv('df26_traffic_time.csv', encoding = 'utf-8', index = False)

# explicitly set it as geometry for the GeoDataFrame
df27_building.set_geometry(col='geometry', inplace=True)
df27_building.to_file('df27_building.geojson', driver="GeoJSON")

df28_grid_building.to_file('df28_grid_building.geojson', driver="GeoJSON")
df29_sports.to_file('df29_sports.geojson', driver="GeoJSON")
df30_academy.to_file('df30_academy.geojson', driver="GeoJSON")